In [7]:
import numpy as np
import tensorflow as tf

# 1. RNN
<br />
### 3개의 연속된 알파벳으로 다음 알파벳이 무엇이 나오는지 학습하고자 함

In [8]:
np.random.seed(7)
char_arr = ['A', 'B', 'C', 'D', 'E', 'F', 'G',
            'H', 'I', 'J', 'K', 'L', 'M', 'N',
            'O', 'P', 'Q', 'R', 'S', 'T', 'U',
            'V', 'W', 'X', 'Y', 'Z']

# one hot enc & decoding을 위한 배열 생성
# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 알파벳 데이터 생성
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

seq_length = 4
data = []


for i in range(0, len(alphabet) - seq_length):
    seq_in = alphabet[i:i + seq_length]
    seq_out = alphabet[i + seq_length-1]
    data.append(seq_in)
num_dic

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

### batch 및 one hot encoding을 위한 함수 정의

In [9]:
#one hot Encoding
def make_batch(seq_data):
    input_batch = []
    target_batch = []

    for seq in seq_data:
        # input_batch 와 target_batch 는 알파벳 배열의 인덱스
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[input])

        target_batch.append(target)

    return input_batch, target_batch

In [10]:
make_batch(data)

([array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
  array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
  array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0

### Hyper Parameter 설정

In [11]:
# Hyper parameter setting
learning_rate = 0.01
n_hidden = 128
total_epoch = 300

# 타입 스텝: [1 2 3] => 3
# RNN 을 구성하는 시퀀스의 갯수
time_step = 3

# 입력값 크기. 알파벳에 대한 one-hot 인코딩이므로 26개
# 예) c => [0 0 1 0 0 0 0 0 0 0 0 ... 0]
# 출력값도 입력값과 마찬가지로 26개의 알파벳으로 분류
n_input = n_class = dic_len

### RNN Layer 생성

In [12]:
tf.reset_default_graph()
# RNN Layer 구성
X = tf.placeholder(tf.float32, [None, time_step, n_input])
Y = tf.placeholder(tf.int32, [None])


W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

# RNN 셀을 생성
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# 과적합 방지를 위한 Dropout
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)

## tf.nn.dynamic_rnn 함수를 이용해 RNN 생성
outputs, states = tf.nn.dynamic_rnn(cell1, X, dtype=tf.float32)


outputs = tf.transpose(outputs, [1, 0, 2])# 최종 결과는 one-hot 인코딩 형식으로 변환
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### 모델 학습

In [9]:
# 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})
    if epoch%10 == 0:
        print('Epoch:', '%04d' % (epoch + 1),
              'cost =', '{:.6f}'.format(loss))

print('Optimized')

Epoch: 0001 cost = 3.541897
Epoch: 0011 cost = 0.409346
Epoch: 0021 cost = 0.038857
Epoch: 0031 cost = 0.024987
Epoch: 0041 cost = 0.017415
Epoch: 0051 cost = 0.014460
Epoch: 0061 cost = 0.001642
Epoch: 0071 cost = 0.000093
Epoch: 0081 cost = 0.000622
Epoch: 0091 cost = 0.001563
Epoch: 0101 cost = 0.000703
Epoch: 0111 cost = 0.000035
Epoch: 0121 cost = 0.000063
Epoch: 0131 cost = 0.000114
Epoch: 0141 cost = 0.000073
Epoch: 0151 cost = 0.000371
Epoch: 0161 cost = 0.000156
Epoch: 0171 cost = 0.000009
Epoch: 0181 cost = 0.000347
Epoch: 0191 cost = 0.000004
Epoch: 0201 cost = 0.000003
Epoch: 0211 cost = 0.000014
Epoch: 0221 cost = 0.000021
Epoch: 0231 cost = 0.000046
Epoch: 0241 cost = 0.007622
Epoch: 0251 cost = 0.000009
Epoch: 0261 cost = 0.000009
Epoch: 0271 cost = 0.000069
Epoch: 0281 cost = 0.000036
Epoch: 0291 cost = 0.000011
Optimized


### 결과 확인

In [10]:
prediction = tf.cast(tf.argmax(model, 1), tf.int32)

prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(data)

predict, accuracy_val = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

predict_words = []
for idx, val in enumerate(data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('Input:', [w[:3] + ' ' for w in data])
print('Predict:', predict_words)

Input: ['ABC ', 'BCD ', 'CDE ', 'DEF ', 'EFG ', 'FGH ', 'GHI ', 'HIJ ', 'IJK ', 'JKL ', 'KLM ', 'LMN ', 'MNO ', 'NOP ', 'OPQ ', 'PQR ', 'QRS ', 'RST ', 'STU ', 'TUV ', 'UVW ', 'VWX ']
Predict: ['ABCD', 'BCDE', 'CDEF', 'DEFG', 'EFGH', 'FGHI', 'GHIJ', 'HIJK', 'IJKL', 'JKLM', 'KLMN', 'LMNO', 'MNOP', 'NOPQ', 'OPQR', 'PQRS', 'QRST', 'RSTU', 'STUV', 'TUVW', 'UVWX', 'VWXY']
